In [57]:
import os
import sys
from datetime import datetime, timedelta
from dateutil.tz import tzlocal

from mastodon import Mastodon

import numpy as np
import pandas as pd

## Data Crawling and Preprocessing

### Login and Test Mastodon API

In [14]:
# Mastodon.create_app(
#     'crawler',
#     api_base_url = 'https://mastodon.social',
#     to_file = 'clientcred.secret'
# )

In [15]:
mastodon = Mastodon(client_id = 'clientcred.secret',)
mastodon.log_in(
    'e0925463@u.nus.edu',
    'social_scope',
    to_file = 'usercred.secret'
)

'SFafhBc-qGn6Dn519u20VWE9dB46APFrpEqYUflr2WQ'

In [5]:
mastodon.status("109355825433550347")

{'id': 109355825433550347,
 'created_at': datetime.datetime(2022, 11, 16, 22, 24, 33, 47000, tzinfo=tzlocal()),
 'in_reply_to_id': None,
 'in_reply_to_account_id': None,
 'sensitive': False,
 'spoiler_text': '',
 'visibility': 'public',
 'language': 'en',
 'uri': 'https://mastodon.social/users/jeffjarvis/statuses/109355825433550347',
 'url': 'https://mastodon.social/@jeffjarvis/109355825433550347',
 'replies_count': 12,
 'reblogs_count': 104,
 'favourites_count': 219,
 'edited_at': None,
 'favourited': False,
 'reblogged': False,
 'muted': False,
 'bookmarked': False,
 'content': '<p><span class="h-card"><a href="https://universeodon.com/@georgetakei" class="u-url mention">@<span>georgetakei</span></a></span> has arrived and he wants us to follow him, which, of course, we should do....</p>',
 'filtered': [],
 'reblog': None,
 'application': {'name': 'Web', 'website': None},
 'account': {'id': 109243139631965273,
  'username': 'jeffjarvis',
  'acct': 'jeffjarvis',
  'display_name': 'Jef

In [ ]:
# Note: maximum limit of status is only 40
statuses = mastodon.account_statuses("108199052370473764", limit=2)

### Crawl User Statuses/Toots Data

In [58]:
user_id = "108199052370473764"

min_date = datetime(2022,5,1,0,0,0, tzinfo=tzlocal())
max_date = datetime(2023,2,1,0,0,0, tzinfo=tzlocal())
curr_date = min_date
done_ind = False

df = pd.DataFrame(columns=['user_id','status_created_datetime','status_id', 'status_uri','status_url','status_content_raw',
                           'reblog',
                           'reblog_user_id','reblog_created_datetime','reblog_id','reblog_uri','reblog_url','reblog_content_raw'])

In [61]:
df_dict_list = []

while not(done_ind):
    statuses = mastodon.account_statuses(user_id, min_id=curr_date, max_id=max_date, limit=40)
    
    if statuses == []:
        curr_date = curr_date + timedelta(days=40)
    else:
        for status in reversed(statuses):
            row_dict = {}
            if status['created_at'] < max_date:
                row_dict = {
                                'user_id': user_id,
                                'status_created_datetime': status['created_at'],
                                'status_id': status['id'], 
                                'status_uri': status['uri'],
                                'status_url': status['url'],
                                'status_content_raw': status['content'],
                                'reblog': False
                }

                if not(status['reblog'] is None):
                    row_dict['reblog'] = True
                    row_dict['reblog_user_id'] = status['reblog']['account']['id']
                    row_dict['reblog_created_datetime'] = status['reblog']['created_at']
                    row_dict['reblog_id'] = status['reblog']['id']
                    row_dict['reblog_uri'] = status['reblog']['uri']
                    row_dict['reblog_url'] = status['reblog']['url']
                    row_dict['reblog_content_raw'] = status['reblog']['content']

                curr_date = status['created_at']

                df_dict_list.append(row_dict)

            else:
                done_ind = True
                break
    
    if len(statuses) < 40:
        done_ind = True
    
    if curr_date >= max_date:
        done_ind = True

In [62]:
df = pd.DataFrame.from_records(df_dict_list)

In [64]:
print(len(df))
df.head(10)

478


,user_id,status_created_datetime,status_id,status_uri,status_url,status_content_raw,reblog,reblog_user_id,reblog_created_datetime,reblog_id,reblog_uri,reblog_url,reblog_content_raw
0,108199052370473764,2022-11-07 18:49:47.011000+00:00,109304020140430484,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109304020...,<p>The wild backstory of how Trump pitted his ...,False,NaN,NaT,NaN,NaN,NaN,NaN
1,108199052370473764,2022-11-07 20:28:14.677000+00:00,109304407301496053,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/users/drewharwell/stat...,,True,1.092146e+17,2022-11-03 21:15:22.678000+00:00,1.092819e+17,https://mastodon.social/users/willoremus/statu...,https://mastodon.social/@willoremus/1092819433...,<p>Wrote today about the mess that Musk has ma...
2,108199052370473764,2022-11-08 20:21:04.844000+00:00,109310041440106644,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109310041...,<p>Buncha BS out there already about the midte...,False,NaN,NaT,NaN,NaN,NaN,NaN
3,108199052370473764,2022-11-09 13:28:03.752000+00:00,109314079702469733,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109314079...,<p>The company formerly known as Facebook is l...,False,NaN,NaT,NaN,NaN,NaN,NaN
4,108199052370473764,2022-11-09 13:34:13.817000+00:00,109314103955775697,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109314103...,<p>164 Republicans who believe Venezuelan vote...,False,NaN,NaT,NaN,NaN,NaN,NaN
5,108199052370473764,2022-11-09 15:05:33.741000+00:00,109314463087688472,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/users/drewharwell/stat...,,True,4.730400e+04,2022-11-09 14:15:50+00:00,1.093143e+17,https://mastodon.xyz/users/brianfagioli/status...,https://mastodon.xyz/@brianfagioli/10931426756...,"<p>A must read from <span class=""h-card""><a hr..."
6,108199052370473764,2022-11-09 22:07:16.113000+00:00,109316121304402012,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109316121...,"<p><span class=""h-card""><a href=""https://masto...",False,NaN,NaT,NaN,NaN,NaN,NaN
7,108199052370473764,2022-11-11 18:09:15.694000+00:00,109326510042994272,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109326510...,"<p><span class=""h-card""><a href=""https://masto...",False,NaN,NaT,NaN,NaN,NaN,NaN
8,108199052370473764,2022-11-14 14:22:06.703000+00:00,109342603785237555,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109342603...,<p>That fake Eli Lilly account on the other we...,False,NaN,NaT,NaN,NaN,NaN,NaN
9,108199052370473764,2022-11-18 16:33:38.799000+00:00,109365770242122117,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/users/drewharwell/stat...,,True,1.093276e+17,2022-11-17 23:22:03+00:00,1.093617e+17,https://infosec.exchange/users/JosephMenn/stat...,https://infosec.exchange/@JosephMenn/109361713...,<p>Just Posted: Musk eases off the gas on retu...


### Tokenizing (a single) Status/Toot - and Remove URL

In [65]:
import spacy
from spacy_html_tokenizer import create_html_tokenizer

import re

In [126]:
nlp = spacy.blank("en")
nlp.tokenizer = create_html_tokenizer()(nlp)
nlp.add_pipe('sentencizer')

sample_html = 'The wild backstory of how Trump pitted his own social media company against another company run by a Trump adviser to get a more lucrative deal for himself: '
doc = nlp(sample_html)

In [128]:
for sent in doc.sents:
    print(re.sub(r'http\S+', '', sent.text))

The wild backstory of how Trump pitted his own social media company against another company run by a Trump adviser to get a more lucrative deal for himself:
156


### Tokenizing Whole Raw Content Data

In [129]:
nlp = spacy.blank("en")
nlp.tokenizer = create_html_tokenizer()(nlp)
nlp.add_pipe('sentencizer')

def html_tokenize(html_text, nlp=nlp):
    res = ""
    if html_text == "" or html_text is None or not(type(html_text)==str):
        return None
    else:
        doc = nlp(html_text)
        for sent in doc.sents:
            res += " " + re.sub(r'http\S+', '', sent.text)
        return res[1:]

In [130]:
status_content = [html_tokenize(text) for text in df['status_content_raw']]
reblog_content = [html_tokenize(text) for text in df['reblog_content_raw']]

df['status_content'] = status_content
df['reblog_content'] = reblog_content

The wild backstory of how Trump pitted his own social media company against another company run by a Trump adviser to get a more lucrative deal for himself: 


In [132]:
df.head()

,user_id,status_created_datetime,status_id,status_uri,status_url,status_content_raw,reblog,reblog_user_id,reblog_created_datetime,reblog_id,reblog_uri,reblog_url,reblog_content_raw,status_content,reblog_content,status_token_count,reblog_token_count
0,108199052370473764,2022-11-07 18:49:47.011000+00:00,109304020140430484,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109304020...,<p>The wild backstory of how Trump pitted his ...,False,NaN,NaT,NaN,NaN,NaN,NaN,The wild backstory of how Trump pitted his own...,None,129,0
1,108199052370473764,2022-11-07 20:28:14.677000+00:00,109304407301496053,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/users/drewharwell/stat...,,True,1.092146e+17,2022-11-03 21:15:22.678000+00:00,1.092819e+17,https://mastodon.social/users/willoremus/statu...,https://mastodon.social/@willoremus/1092819433...,<p>Wrote today about the mess that Musk has ma...,The wild backstory of how Trump pitted his own...,None,129,0
2,108199052370473764,2022-11-08 20:21:04.844000+00:00,109310041440106644,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109310041...,<p>Buncha BS out there already about the midte...,False,NaN,NaT,NaN,NaN,NaN,NaN,The wild backstory of how Trump pitted his own...,None,129,0
3,108199052370473764,2022-11-09 13:28:03.752000+00:00,109314079702469733,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109314079...,<p>The company formerly known as Facebook is l...,False,NaN,NaT,NaN,NaN,NaN,NaN,The wild backstory of how Trump pitted his own...,None,129,0
4,108199052370473764,2022-11-09 13:34:13.817000+00:00,109314103955775697,https://mastodon.social/users/drewharwell/stat...,https://mastodon.social/@drewharwell/109314103...,<p>164 Republicans who believe Venezuelan vote...,False,NaN,NaT,NaN,NaN,NaN,NaN,The wild backstory of how Trump pitted his own...,None,129,0


## Fine Tuning GPT3

### Setup OpenAI API

In [134]:
import openai

openai.api_key = ""
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
# Test
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2020 Summer Olympics men's high jump?
A:"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

### Calculate Embedding

In [ ]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [ ]:
document_embeddings = compute_doc_embeddings(df)

### Finding Similar Documents by Embedding

In [ ]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities